In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')
path="../../../ukb/data/"

'''
Helper function to find out columns containing specific text
'''
def findcols(df,text):
    return [col for col in df.columns if text in col]

In [2]:
'''
set of variables removed as noise
'''

drop_ohc_cols=['death_date','age_at_death_f40007', 'date_of_death_f40000', 'death_record_format_f40018',
 'death_record_origin_f40020', 'description_of_cause_of_death_f40010',
'date_of_alzheimers_disease_report_f42020', 'source_of_alzheimers_disease_report_f42021',
'date_of_all_cause_dementia_report_f42018', 'date_of_frontotemporal_dementia_report_f42024', 'date_of_vascular_dementia_report_f42022',
 'source_of_all_cause_dementia_report_f42019', 'source_of_frontotemporal_dementia_report_f42025', 
 'source_of_vascular_dementia_report_f42023','Age_Today']

exc_ohc_cols=['eid','death']

disease_vars=['EPIL_bef','DIAB_bef','CERVASC_bef','CERVASCALL_bef','STROKE_bef','AD','EPIL','DIAB','CERVASC','CELIAC','PD']

In [5]:
qualmap={'None of the above':0,'Prefer not to answer':0,'O levels/GCSEs or equivalent':1,\
'NVQ or HND or HNC or equivalent':2,'NVQ or HND or HNC or equivalent' :3,\
'NVQ or HND or HNC or equivalent':4,'CSEs or equivalent':5,\
'Other professional qualifications eg: nursing, teaching':6,'A levels/AS levels or equivalent':7,\
'College or University degree':8 }   


In [6]:
def onehotencoder(df,dropcols,exccols,maxrecs=12,mincount=0.8,minctcts=0.8,nodrop=0):
    
    if nodrop==0:
        df1=df.drop(columns=np.append(dropcols,exccols),axis=0)
    else:
        df1=df.copy()
   
    
    print(r'Total variables = %0.0f ' % (df1.shape[1]-1))
    
    ohe_cols=\
    [col for col in df1.columns if len(df1[col].value_counts())<maxrecs and df1[col].count()/df1[col].shape[0]>mincount]
    
    print(r'Total ohe variables = %0.0f ' % (len(ohe_cols)))
        
    df_ohe_cols=df1[ohe_cols]
    
    cts_cols=[col for i,col in enumerate(df1.columns) if re.search(str(df1.dtypes[i]),'float64|int')
             #and len(df1[col].value_counts())>=maxrecs 
              and 
              df1[col].count()/len(df1[col])>minctcts]
    
    print(r'Total cts variables = %0.0f ' % (len(cts_cols)))
    
    df_ctsvars=df1[cts_cols]
    
    df_ctsvars['eid']=df['eid']
    
    for col in df_ohe_cols.columns:
        df_ohe_cols[col][pd.isnull(df_ohe_cols[col])]='Unknown'
    df_out=pd.get_dummies(df_ohe_cols, prefix=df_ohe_cols.columns,
               columns=df_ohe_cols.columns)
    
    print(r'Total ohe variables post trans = %0.0f ' % (df_out.shape[1]-1))
    
    exccols2=[col for col in df.columns if col in exccols]
    
    df_out[exccols2]=df[exccols2]
    
    df_full=pd.merge(df_out,df_ctsvars,on='eid',how='left')    
    
    
    print(r'Total variables out = %0.0f ' % (df_full.shape[1]-1))
    
    return df_full

In [7]:
ukb_tp0=pd.read_pickle('%s%s' % (path,'ukb_tp0_.p'))

In [8]:
ukb_tp0['qualif_score']=ukb_tp0['qualifications_f6138'].map(qualmap)
ukb_tp0['qualif_score'][pd.isnull(ukb_tp0['qualif_score'])]=0


In [9]:
ukb_tp0_ohc=onehotencoder(ukb_tp0,dropcols=drop_ohc_cols,exccols=exc_ohc_cols,maxrecs=9,mincount=0.8)

Total variables = 1505 
Total ohe variables = 358 
Total cts variables = 189 
Total ohe variables post trans = 1464 
Total variables out = 1655 


In [10]:
ukb_tp0_ohc.to_pickle('%s%s' % (path,'ukb_tp0_ohc_.p'))